# Experiments

### Setup

In [1]:
# Or you can use a .env file
from dotenv import load_dotenv
load_dotenv(dotenv_path="../../.env", override=True)

True

Here is the RAG Application that we've been working with throughout this course

In [2]:
import os
import tempfile
from langchain.text_splitter import RecursiveCharacterTextSplitter
from langchain_community.document_loaders.sitemap import SitemapLoader
from langchain_community.vectorstores import SKLearnVectorStore
from langchain_openai import OpenAIEmbeddings
from langsmith import traceable
from openai import OpenAI
from typing import List
import nest_asyncio

# TODO: Configure this model!
MODEL_NAME = "gpt-4o"
MODEL_PROVIDER = "openai"
APP_VERSION = 1.0
RAG_SYSTEM_PROMPT = """You are an assistant for question-answering tasks. 
Use the following pieces of retrieved context to answer the latest question in the conversation. 
If you don't know the answer, just say that you don't know. 
Use three sentences maximum and keep the answer concise.
"""

openai_client = OpenAI()

def get_vector_db_retriever():
    persist_path = os.path.join(tempfile.gettempdir(), "union.parquet")
    embd = OpenAIEmbeddings()

    # If vector store exists, then load it
    if os.path.exists(persist_path):
        vectorstore = SKLearnVectorStore(
            embedding=embd,
            persist_path=persist_path,
            serializer="parquet"
        )
        return vectorstore.as_retriever(lambda_mult=0)

    # Otherwise, index LangSmith documents and create new vector store
    ls_docs_sitemap_loader = SitemapLoader(web_path="https://docs.smith.langchain.com/sitemap.xml")
    ls_docs = ls_docs_sitemap_loader.load()

    text_splitter = RecursiveCharacterTextSplitter.from_tiktoken_encoder(
        chunk_size=500, chunk_overlap=0
    )
    doc_splits = text_splitter.split_documents(ls_docs)

    vectorstore = SKLearnVectorStore.from_documents(
        documents=doc_splits,
        embedding=embd,
        persist_path=persist_path,
        serializer="parquet"
    )
    vectorstore.persist()
    return vectorstore.as_retriever(lambda_mult=0)

nest_asyncio.apply()
retriever = get_vector_db_retriever()

"""
retrieve_documents
- Returns documents fetched from a vectorstore based on the user's question
"""
@traceable(run_type="chain")
def retrieve_documents(question: str):
    return retriever.invoke(question)

"""
generate_response
- Calls `call_openai` to generate a model response after formatting inputs
"""
@traceable(run_type="chain")
def generate_response(question: str, documents):
    formatted_docs = "\n\n".join(doc.page_content for doc in documents)
    messages = [
        {
            "role": "system",
            "content": RAG_SYSTEM_PROMPT
        },
        {
            "role": "user",
            "content": f"Context: {formatted_docs} \n\n Question: {question}"
        }
    ]
    return call_openai(messages)

"""
call_openai
- Returns the chat completion output from OpenAI
"""
@traceable(
    run_type="llm",
    metadata={
        "ls_provider": MODEL_PROVIDER,
        "ls_model_name": MODEL_NAME
    }
)
def call_openai(messages: List[dict]) -> str:
    return openai_client.chat.completions.create(
        model=MODEL_NAME,
        messages=messages,
    )

"""
langsmith_rag
- Calls `retrieve_documents` to fetch documents
- Calls `generate_response` to generate a response based on the fetched documents
- Returns the model response
"""
@traceable(run_type="chain")
def langsmith_rag(question: str):
    documents = retrieve_documents(question)
    response = generate_response(question, documents)
    return response.choices[0].message.content


USER_AGENT environment variable not set, consider setting it to identify your requests.


### Experiment

Here is a code snippet that should look similar to what you see from the starter code!

There are a few important components here.

1. We have defined an Evaluator
2. We pipe our dataset examples (dict) to the shape of input that our function `langsmith_rag` takes (str) using a target function

In [3]:
from langsmith import evaluate, Client

client = Client()
dataset_name = "RAG Application Golden Dataset"

def is_concise_enough(reference_outputs: dict, outputs: dict) -> dict:
    score = len(outputs["output"]) < 1.5 * len(reference_outputs["output"])
    return {"key": "is_concise", "score": int(score)}

def target_function(inputs: dict):
    return langsmith_rag(inputs["question"])

evaluate(
    target_function,
    data=dataset_name,
    evaluators=[is_concise_enough],
    experiment_prefix="gpt-4o"
)

/Users/bluebird/develop/intro-to-langsmith/ls-academy/lib/python3.10/site-packages/tqdm/auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


View the evaluation results for experiment: 'gpt-4o-f53cdce0' at:
https://smith.langchain.com/o/176b8476-3bdc-40b7-b3e3-1a9621f6f957/datasets/7edf5680-d311-4654-ad30-05a4ee45e5e7/compare?selectedSessions=235476ce-b14c-49dc-8c8f-f247e4600ef7




15it [00:28,  1.93s/it]


,inputs.question,outputs.output,error,reference.output,feedback.is_concise,execution_time,example_id,id
0,How do I pass metadata in with @traceable?,To pass metadata using the `@traceable` decora...,None,You can pass metadata with the @traceable deco...,1,2.369644,107eb36b-a1d1-496b-b45d-977fb0a0a8ec,dd1eb51a-1b5d-4baf-8976-50f8cfb6aed8
1,How do I set up tracing to LangSmith if I'm us...,To set up tracing to LangSmith using LangChain...,None,"To set up tracing to LangSmith with LangChain,...",1,1.668975,1bc6217d-8669-4dc8-a826-063ca9d45acd,2fb71e15-e4df-4f4a-8387-420c6baece0c
2,What programming languages does LangSmith SDK ...,The LangSmith SDK supports Python and TypeScript.,None,The LangSmith SDK supports multiple programmin...,1,0.804488,440ffd98-da20-4154-b44e-c58dc4415a6c,789905ce-5ce1-4089-9ee4-be0646e429e8
3,Can LangSmith be used for finetuning and model...,"No, LangSmith is not designed for finetuning a...",None,"Yes, LangSmith can be used for fine-tuning and...",1,1.666284,82509e8f-eaad-4469-90fb-922862bb55f3,644aaa90-9541-4bed-ac9c-770dde68ea99
4,Does LangSmith support offline evaluation?,"Yes, LangSmith supports offline evaluation. Yo...",None,"Yes, LangSmith supports offline evaluation thr...",1,1.178987,c5400b9a-7d45-4882-84d1-d4e160d4667d,c03dc311-8b38-4c5f-a1fd-4f20651239ed
5,Can I integrate LangSmith with other machine l...,The provided context does not specify whether ...,None,"Yes, LangSmith can be integrated with various ...",1,1.229385,a47f8f17-dfac-41b0-b3e6-5dfa2773ca67,219de47b-bbd4-490b-bcf9-6cbf879fc194
6,What are the main features of LangSmith?,The main features of LangSmith include tracing...,None,LangSmith offers a variety of features includi...,1,2.189403,a5df4662-6c4c-456d-afaa-2461e6b4bc4a,a5557f0c-54c4-4075-8e21-a991e8dadeab
7,Is there a Javascript Langsmith SDK?,"Yes, there is a JavaScript LangSmith SDK, as m...",None,"Yes, there is a Javascript Langsmith SDK.",0,4.287467,6518339e-b55e-4ad8-846d-8cd4ee8afca9,3272d4e0-2e32-4712-9fbc-4039a9270ce2
8,How do I create user feedback with the LangSmi...,To create user feedback with the LangSmith SDK...,None,To create user feedback with the LangSmith SDK...,1,1.787151,afa301e2-d2e9-4407-a421-3bc1b5f1b4e0,2b3014a0-3ff3-44c4-b243-56f11cdb67df
9,Can LangSmith be used to evaluate agents?,"Yes, LangSmith can be used to evaluate agents....",None,"Yes, LangSmith can be used to evaluate agents....",1,1.581269,52a86e00-73a8-4416-b444-772adbe3e0c1,aafa1829-f2cb-4e17-896a-86f33d0853a8


### Modifying your Application

Now, let's change our model to gpt-35-turbo and see how it performs!

Make this change, and then run this code snippet!

In [4]:
from langsmith import evaluate, Client
from langsmith.schemas import Example, Run

def target_function(inputs: dict):
    return langsmith_rag(inputs["question"])

evaluate(
    target_function,
    data=dataset_name,
    evaluators=[is_concise_enough],
    experiment_prefix="gpt-3.5-turbo"
)

View the evaluation results for experiment: 'gpt-3.5-turbo-ac4e4739' at:
https://smith.langchain.com/o/176b8476-3bdc-40b7-b3e3-1a9621f6f957/datasets/7edf5680-d311-4654-ad30-05a4ee45e5e7/compare?selectedSessions=11c8c9d1-781e-414e-80f4-19daa6807559




15it [00:26,  1.80s/it]


,inputs.question,outputs.output,error,reference.output,feedback.is_concise,execution_time,example_id,id
0,How do I pass metadata in with @traceable?,To pass metadata with the `@traceable` decorat...,None,You can pass metadata with the @traceable deco...,1,2.382028,107eb36b-a1d1-496b-b45d-977fb0a0a8ec,f5e1dd70-dcba-4578-ba90-404caf63a977
1,How do I set up tracing to LangSmith if I'm us...,To set up tracing to LangSmith using LangChain...,None,"To set up tracing to LangSmith with LangChain,...",1,2.199156,1bc6217d-8669-4dc8-a826-063ca9d45acd,8d7543a2-09b2-42a4-b1ec-765e1aa8335d
2,What programming languages does LangSmith SDK ...,LangSmith SDK supports Python and TypeScript.,None,The LangSmith SDK supports multiple programmin...,1,0.794253,440ffd98-da20-4154-b44e-c58dc4415a6c,36a07a0a-2ae8-49ec-b5bd-23501c3c52c4
3,Can LangSmith be used for finetuning and model...,LangSmith is primarily focused on LLM applicat...,None,"Yes, LangSmith can be used for fine-tuning and...",1,2.254451,82509e8f-eaad-4469-90fb-922862bb55f3,53ec2f01-ca71-4d17-b111-63d72a9f0ca1
4,Does LangSmith support offline evaluation?,"Yes, LangSmith supports offline evaluation. Yo...",None,"Yes, LangSmith supports offline evaluation thr...",1,0.968235,c5400b9a-7d45-4882-84d1-d4e160d4667d,69ca5d4b-b54f-44e0-bd25-b3aa23c989ae
5,Can I integrate LangSmith with other machine l...,The provided context does not mention specific...,None,"Yes, LangSmith can be integrated with various ...",0,1.903319,a47f8f17-dfac-41b0-b3e6-5dfa2773ca67,78a2be8d-fba1-4a41-b998-4428a2daf5bc
6,What are the main features of LangSmith?,LangSmith's main features include tracing LLM ...,None,LangSmith offers a variety of features includi...,1,1.798798,a5df4662-6c4c-456d-afaa-2461e6b4bc4a,0ea18bfe-4fb3-4dbc-b0ff-39eadeddd8d8
7,Is there a Javascript Langsmith SDK?,"Yes, there is a JavaScript LangSmith SDK.",None,"Yes, there is a Javascript Langsmith SDK.",1,0.744207,6518339e-b55e-4ad8-846d-8cd4ee8afca9,493a04b1-b282-4d22-bb51-297dc38b0209
8,How do I create user feedback with the LangSmi...,To create user feedback with the LangSmith SDK...,None,To create user feedback with the LangSmith SDK...,1,2.108840,afa301e2-d2e9-4407-a421-3bc1b5f1b4e0,e875a38e-1d8c-44bc-8f52-f4531e82a4eb
9,Can LangSmith be used to evaluate agents?,"Yes, LangSmith can be used to evaluate agents....",None,"Yes, LangSmith can be used to evaluate agents....",1,1.179157,52a86e00-73a8-4416-b444-772adbe3e0c1,d46917c6-f7ca-420b-a753-633d7b4e7877


### Running over Different pieces of Data

##### Dataset Version

You can execute an experiment on a specific version of a dataset in the sdk by using the `as_of` parameter in `list_examples`

Let's try running on just our initial dataset.

In [5]:
evaluate(
    target_function,
    data=client.list_examples(dataset_name=dataset_name, as_of="initial dataset"),   # We use as_of to specify a version
    evaluators=[is_concise_enough],
    experiment_prefix="initial dataset version"
)

View the evaluation results for experiment: 'initial dataset version-725c21a2' at:
https://smith.langchain.com/o/176b8476-3bdc-40b7-b3e3-1a9621f6f957/datasets/7edf5680-d311-4654-ad30-05a4ee45e5e7/compare?selectedSessions=7f2202a3-d296-4679-8f8f-ff1a0515e284




10it [00:20,  2.07s/it]


,inputs.question,outputs.output,error,reference.output,feedback.is_concise,execution_time,example_id,id
0,How do I create user feedback with the LangSmi...,To create user feedback with the LangSmith SDK...,None,To create user feedback with the LangSmith SDK...,0,2.891064,afa301e2-d2e9-4407-a421-3bc1b5f1b4e0,4d31bf7a-e1dd-43c0-86cf-d880087a4b7d
1,Can LangSmith be used to evaluate agents?,"Yes, LangSmith can be used to evaluate agents....",None,"Yes, LangSmith can be used to evaluate agents....",1,1.233780,52a86e00-73a8-4416-b444-772adbe3e0c1,846eb6e5-3c2c-416b-9371-8caff75ccd32
2,Can LangSmith be used for finetuning and model...,"No, LangSmith is not designed for finetuning a...",None,"Yes, LangSmith can be used for fine-tuning and...",1,1.675356,82509e8f-eaad-4469-90fb-922862bb55f3,6b3defd9-2677-4563-b9a6-a200a2e4cb8a
3,Does LangSmith support offline evaluation?,"Yes, LangSmith supports offline evaluation. Us...",None,"Yes, LangSmith supports offline evaluation thr...",1,1.144894,c5400b9a-7d45-4882-84d1-d4e160d4667d,e99f2a5f-514d-4eba-917b-dda912eee329
4,Does LangSmith support online evaluation?,"Yes, LangSmith supports online evaluation. You...",None,"Yes, LangSmith supports online evaluation as a...",1,1.310057,e20b4c56-f50e-47d6-aa7a-cf9de22f63ba,510d239e-b005-46d8-8f96-69f7ec677965
5,What testing capabilities does LangSmith have?,LangSmith offers capabilities for creating dat...,None,LangSmith offers capabilities for creating dat...,1,2.334194,37b0995f-290e-4b00-bb7e-f62dfbc90137,302b3143-6ea8-47d3-8ca9-1d033f695324
6,What is LangSmith used for in three sentences?,LangSmith is a platform designed for developin...,None,LangSmith is a platform designed for the devel...,1,3.204441,56eefef7-bdd3-46bd-8284-1af5cdbc6932,8b938164-1a1c-47c3-8169-f31c56680cb4
7,How do I pass metadata in with @traceable?,To pass metadata using the `@traceable` decora...,None,You can pass metadata with the @traceable deco...,1,1.880219,107eb36b-a1d1-496b-b45d-977fb0a0a8ec,88cdd7e5-d03e-4838-8340-fc1ed93220fc
8,How can I trace with the @traceable decorator?,"To trace with the @traceable decorator, you si...",None,To trace with the @traceable decorator in Pyth...,1,2.109586,71585ab2-4011-4dbe-9d9e-af1c6bc593d0,8a06b3e5-8edf-4c76-a3dd-bf135e17ec98
9,How do I set up tracing to LangSmith if I'm us...,To set up tracing to LangSmith using LangChain...,None,To set up tracing to LangSmith while using Lan...,0,2.342540,97641009-0d46-45a4-942d-c263b4306294,c70fcb5c-75a7-4756-b2d1-b263aca21626


##### Dataset Split

You can run an experiment on a specific split of your dataset, let's try running on the Crucial Examples split.

In [6]:
evaluate(
    target_function,
    data=client.list_examples(dataset_name=dataset_name, splits=["crucial examples"]),  # We pass in a list of Splits
    evaluators=[is_concise_enough],
    experiment_prefix="Crucial Examples split"
)

View the evaluation results for experiment: 'Crucial Examples split-c662efee' at:
https://smith.langchain.com/o/176b8476-3bdc-40b7-b3e3-1a9621f6f957/datasets/7edf5680-d311-4654-ad30-05a4ee45e5e7/compare?selectedSessions=b4ff741b-c10c-4cbd-9a66-f36390604b51




5it [00:08,  1.69s/it]


,inputs.question,outputs.output,error,reference.output,feedback.is_concise,execution_time,example_id,id
0,How do I pass metadata in with @traceable?,To pass metadata with `@traceable` in LangSmit...,None,You can pass metadata with the @traceable deco...,1,2.382362,107eb36b-a1d1-496b-b45d-977fb0a0a8ec,a4dde624-5423-4fdb-be1c-fb6f3ecb2912
1,How do I set up tracing to LangSmith if I'm us...,To set up tracing to LangSmith using LangChain...,None,"To set up tracing to LangSmith with LangChain,...",1,1.698342,1bc6217d-8669-4dc8-a826-063ca9d45acd,dc3d3173-420c-4b39-833f-4643f2fdf621
2,What programming languages does LangSmith SDK ...,LangSmith SDK supports integrations in Python ...,None,The LangSmith SDK supports multiple programmin...,1,0.963533,440ffd98-da20-4154-b44e-c58dc4415a6c,6abf38f8-a89e-420d-b043-348bed224cbe
3,Can LangSmith be used for finetuning and model...,"No, LangSmith is not designed for finetuning a...",None,"Yes, LangSmith can be used for fine-tuning and...",1,1.664140,82509e8f-eaad-4469-90fb-922862bb55f3,1bc7e32d-ff5c-4e8c-9112-9f042874fe38
4,Does LangSmith support offline evaluation?,"Yes, LangSmith supports offline evaluation. Yo...",None,"Yes, LangSmith supports offline evaluation thr...",1,1.431586,c5400b9a-7d45-4882-84d1-d4e160d4667d,3a4ac4e7-3240-46f4-a6c4-bd20ae17d8cc


##### Specific Data Points

You can specify individual data points to run an experiment over as well

In [7]:
evaluate(
    target_function,
    data=client.list_examples(
        dataset_name=dataset_name, 
        example_ids=[   # We pass in a specific list of example_ids
            # TODO: You will need to paste in your own example ids for this to work!
            "97641009-0d46-45a4-942d-c263b4306294",
            "afa301e2-d2e9-4407-a421-3bc1b5f1b4e0"
        ]
    ),
    evaluators=[is_concise_enough],
    experiment_prefix="two specific example ids"
)

View the evaluation results for experiment: 'two specific example ids-f48ea47a' at:
https://smith.langchain.com/o/176b8476-3bdc-40b7-b3e3-1a9621f6f957/datasets/7edf5680-d311-4654-ad30-05a4ee45e5e7/compare?selectedSessions=e03c64c8-e62d-4304-a5fd-41af8bdd87a1




2it [00:05,  2.87s/it]


,inputs.question,outputs.output,error,reference.output,feedback.is_concise,execution_time,example_id,id
0,How do I set up tracing to LangSmith if I'm us...,To set up tracing to LangSmith when using Lang...,None,To set up tracing to LangSmith while using Lan...,0,2.213261,97641009-0d46-45a4-942d-c263b4306294,b29a8d7e-0702-4c04-a4bb-921045ac14cf
1,How do I create user feedback with the LangSmi...,You can create user feedback with the LangSmit...,None,To create user feedback with the LangSmith SDK...,1,3.085361,afa301e2-d2e9-4407-a421-3bc1b5f1b4e0,9b40ca54-9809-496f-898f-a28e67a290f2


### Other Parameters

##### Repetitions

You can run an experiment several times to make sure you have consistent results

In [8]:
evaluate(
    target_function,
    data=dataset_name,
    evaluators=[is_concise_enough],
    experiment_prefix="two repetitions",
    num_repetitions=2   # This field defaults to 1
)

View the evaluation results for experiment: 'two repetitions-3e0bc92e' at:
https://smith.langchain.com/o/176b8476-3bdc-40b7-b3e3-1a9621f6f957/datasets/7edf5680-d311-4654-ad30-05a4ee45e5e7/compare?selectedSessions=b104d160-9b01-493d-aaed-04bbc4ea560a




30it [00:56,  1.88s/it]


,inputs.question,outputs.output,error,reference.output,feedback.is_concise,execution_time,example_id,id
0,How do I pass metadata in with @traceable?,"To pass metadata with @traceable in LangSmith,...",None,You can pass metadata with the @traceable deco...,1,2.418413,107eb36b-a1d1-496b-b45d-977fb0a0a8ec,dfed1e9f-4257-41cc-9ca6-2359279946c2
1,How do I set up tracing to LangSmith if I'm us...,"To set up tracing to LangSmith with LangChain,...",None,"To set up tracing to LangSmith with LangChain,...",1,1.709410,1bc6217d-8669-4dc8-a826-063ca9d45acd,ec4fad20-f9b0-4438-aeb7-3edeabaadc5c
2,What programming languages does LangSmith SDK ...,LangSmith SDK supports Python and TypeScript.,None,The LangSmith SDK supports multiple programmin...,1,0.796597,440ffd98-da20-4154-b44e-c58dc4415a6c,fa8e402e-4c21-48cc-8515-21b8ebf70401
3,Can LangSmith be used for finetuning and model...,The provided context does not mention LangSmit...,None,"Yes, LangSmith can be used for fine-tuning and...",1,2.255158,82509e8f-eaad-4469-90fb-922862bb55f3,cb847c17-8d51-4042-870c-745df1b83670
4,Does LangSmith support offline evaluation?,"Yes, LangSmith supports offline evaluation. Yo...",None,"Yes, LangSmith supports offline evaluation thr...",1,1.311029,c5400b9a-7d45-4882-84d1-d4e160d4667d,8d89fc0d-0e51-4126-86d2-fbd0247dd45d
5,Can I integrate LangSmith with other machine l...,The provided context does not specify informat...,None,"Yes, LangSmith can be integrated with various ...",1,1.358486,a47f8f17-dfac-41b0-b3e6-5dfa2773ca67,5858247f-78af-470a-8e3a-1b868a363286
6,What are the main features of LangSmith?,The main features of LangSmith include tracing...,None,LangSmith offers a variety of features includi...,0,2.958910,a5df4662-6c4c-456d-afaa-2461e6b4bc4a,1f83bd4e-bab9-4b09-a441-416e4f0ece33
7,Is there a Javascript Langsmith SDK?,"Yes, there is a JavaScript Langsmith SDK, as i...",None,"Yes, there is a Javascript Langsmith SDK.",0,1.209399,6518339e-b55e-4ad8-846d-8cd4ee8afca9,22ba5251-bcb6-40fe-860b-fac57defd036
8,How do I create user feedback with the LangSmi...,To create user feedback with the LangSmith SDK...,None,To create user feedback with the LangSmith SDK...,1,2.469721,afa301e2-d2e9-4407-a421-3bc1b5f1b4e0,4617af30-e44b-46fd-a598-2d47fe5c973e
9,Can LangSmith be used to evaluate agents?,"Yes, LangSmith can be used to evaluate agents....",None,"Yes, LangSmith can be used to evaluate agents....",1,1.736755,52a86e00-73a8-4416-b444-772adbe3e0c1,cf49532a-21fe-4b2a-99c7-518a77ba12f9


##### Concurrency
You can also kick off concurrent threads of execution to make your experiments finish faster!

In [9]:
evaluate(
    target_function,
    data=dataset_name,
    evaluators=[is_concise_enough],
    experiment_prefix="concurrency",
    max_concurrency=3,  # This defaults to None, so this is an improvement!
)

View the evaluation results for experiment: 'concurrency-4099e766' at:
https://smith.langchain.com/o/176b8476-3bdc-40b7-b3e3-1a9621f6f957/datasets/7edf5680-d311-4654-ad30-05a4ee45e5e7/compare?selectedSessions=4d716c01-b528-463c-921d-bad242053319




15it [00:10,  1.38it/s]


,inputs.question,outputs.output,error,reference.output,feedback.is_concise,execution_time,example_id,id
0,What programming languages does LangSmith SDK ...,LangSmith SDK supports both Python and TypeScr...,None,The LangSmith SDK supports multiple programmin...,1,0.833375,440ffd98-da20-4154-b44e-c58dc4415a6c,0ab7566f-d5c0-489f-af89-8d81bbd1abd5
1,How do I set up tracing to LangSmith if I'm us...,"To set up tracing to LangSmith with LangChain,...",None,"To set up tracing to LangSmith with LangChain,...",1,1.935903,1bc6217d-8669-4dc8-a826-063ca9d45acd,a219a4d4-38e9-455a-85d1-89a888e16dd7
2,How do I pass metadata in with @traceable?,"To pass metadata with `@traceable`, you can sp...",None,You can pass metadata with the @traceable deco...,1,2.321261,107eb36b-a1d1-496b-b45d-977fb0a0a8ec,736e29a0-30ba-4207-bc3d-e5255100cf4d
3,Does LangSmith support offline evaluation?,"Yes, LangSmith supports offline evaluation by ...",None,"Yes, LangSmith supports offline evaluation thr...",1,0.862829,c5400b9a-7d45-4882-84d1-d4e160d4667d,090d3d60-2c6d-4c2d-aefa-e2f18d6d8b0b
4,Can I integrate LangSmith with other machine l...,The provided context does not mention specific...,None,"Yes, LangSmith can be integrated with various ...",1,0.838742,a47f8f17-dfac-41b0-b3e6-5dfa2773ca67,737a7521-454d-4b12-8a7b-ea4d5e2c7669
5,Can LangSmith be used for finetuning and model...,"No, LangSmith is not used for finetuning and m...",None,"Yes, LangSmith can be used for fine-tuning and...",1,2.505060,82509e8f-eaad-4469-90fb-922862bb55f3,d8bcaa60-6ef6-4da4-a088-bdba433e35cb
6,Is there a Javascript Langsmith SDK?,"Yes, there is a LangSmith SDK for JavaScript/T...",None,"Yes, there is a Javascript Langsmith SDK.",1,0.796552,6518339e-b55e-4ad8-846d-8cd4ee8afca9,0da37efb-7fa7-45e0-af31-f95029d83213
7,Can LangSmith be used to evaluate agents?,"Yes, LangSmith can be used to evaluate agents....",None,"Yes, LangSmith can be used to evaluate agents....",1,1.036214,52a86e00-73a8-4416-b444-772adbe3e0c1,9d4b349f-9df1-4ae4-973a-2d8b8b28cadc
8,How do I create user feedback with the LangSmi...,To create user feedback with the LangSmith SDK...,None,To create user feedback with the LangSmith SDK...,1,1.677091,afa301e2-d2e9-4407-a421-3bc1b5f1b4e0,ca7d491d-eade-4c0e-80d6-e3e8e3f756c5
9,What are the main features of LangSmith?,The main features of LangSmith include tracing...,None,LangSmith offers a variety of features includi...,0,3.408403,a5df4662-6c4c-456d-afaa-2461e6b4bc4a,ae6bb1ba-86f8-4ccd-8560-d1dbf6ecdddd


##### Metadata 

You can (and should) add metadata to your experiments, to make them easier to find in the UI

In [10]:
evaluate(
    target_function,
    data=dataset_name,
    evaluators=[is_concise_enough],
    experiment_prefix="metadata added",
    metadata={  # We can pass custom metadata for the experiment, such as the model name
        "model_name": MODEL_NAME
    }
)

View the evaluation results for experiment: 'metadata added-bbc62d80' at:
https://smith.langchain.com/o/176b8476-3bdc-40b7-b3e3-1a9621f6f957/datasets/7edf5680-d311-4654-ad30-05a4ee45e5e7/compare?selectedSessions=cf43f6d4-d6ab-4e1b-a696-d28c4f40efd2




15it [00:33,  2.21s/it]


,inputs.question,outputs.output,error,reference.output,feedback.is_concise,execution_time,example_id,id
0,How do I pass metadata in with @traceable?,"To pass metadata with `@traceable`, you should...",None,You can pass metadata with the @traceable deco...,1,1.520137,107eb36b-a1d1-496b-b45d-977fb0a0a8ec,5fe125fb-e5b8-413a-8c33-e620a30b2ae2
1,How do I set up tracing to LangSmith if I'm us...,To set up tracing to LangSmith when using Lang...,None,"To set up tracing to LangSmith with LangChain,...",1,2.051492,1bc6217d-8669-4dc8-a826-063ca9d45acd,a4c791e6-425d-4f36-b74c-1efd86be960d
2,What programming languages does LangSmith SDK ...,LangSmith SDK supports Python and TypeScript.,None,The LangSmith SDK supports multiple programmin...,1,0.673963,440ffd98-da20-4154-b44e-c58dc4415a6c,8bbf04ab-c910-4a91-9fd4-f39835beaacf
3,Can LangSmith be used for finetuning and model...,"No, LangSmith is not specifically designed for...",None,"Yes, LangSmith can be used for fine-tuning and...",1,2.609392,82509e8f-eaad-4469-90fb-922862bb55f3,38dbdad7-0948-40bb-bb8c-488f4bbe6a7f
4,Does LangSmith support offline evaluation?,"Yes, LangSmith supports offline evaluation by ...",None,"Yes, LangSmith supports offline evaluation thr...",1,3.217731,c5400b9a-7d45-4882-84d1-d4e160d4667d,815145c0-33dd-400d-a076-5747ba455a27
5,Can I integrate LangSmith with other machine l...,The provided context does not specify whether ...,None,"Yes, LangSmith can be integrated with various ...",1,0.932713,a47f8f17-dfac-41b0-b3e6-5dfa2773ca67,8711375e-c5ea-4bd7-a1e6-c614e02be239
6,What are the main features of LangSmith?,LangSmith's main features include tracing LLM ...,None,LangSmith offers a variety of features includi...,0,3.473766,a5df4662-6c4c-456d-afaa-2461e6b4bc4a,fa30369d-7350-47fb-accd-92ee814673f7
7,Is there a Javascript Langsmith SDK?,"Yes, there is a JavaScript LangSmith SDK.",None,"Yes, there is a Javascript Langsmith SDK.",1,0.861961,6518339e-b55e-4ad8-846d-8cd4ee8afca9,03d9c1e9-867f-4a2f-933a-0b614c0c7a52
8,How do I create user feedback with the LangSmi...,To create user feedback with the LangSmith SDK...,None,To create user feedback with the LangSmith SDK...,0,3.332862,afa301e2-d2e9-4407-a421-3bc1b5f1b4e0,07a1aa10-215a-43e0-9f72-48187483b394
9,Can LangSmith be used to evaluate agents?,"Yes, LangSmith can be used to evaluate agents....",None,"Yes, LangSmith can be used to evaluate agents....",1,0.933489,52a86e00-73a8-4416-b444-772adbe3e0c1,f182946d-a456-4c5c-b64d-107720ba7d39
